In [1]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

from plotly.subplots import make_subplots
from fludarabine import model_helper 
from fludarabine.helper import *
from fludarabine.plotly_helper import *

In [2]:
# Input patient information
patient_id = 'GCLL-0257'
wbc_file = '../../../data/gcll/CLL8 CLL cell count 05222025 UPDATED.csv'
treatment_file = '../../../data/gcll/GCLL_treatment.txt'
UMI_start = 1
UMI_end = 4  # MRD3 
# 
treatment_start = 0


In [3]:
# Manually verify the sample list order is correct

In [4]:
sample_list = ['GCLL-0257-T-01',
       'RP-1895_PDO-33508_SM-NKJZN_GCLL-0257-MRD-01_v1_Custom_Selection_OnPrem','RP-1895_PDO-33508_SM-NKJZV_GCLL-0257-MRD-03_v1_Custom_Selection_OnPrem','RP-1895_PDO-33508_SM-NKK1M_GCLL-0257-MRD-04_v1_Custom_Selection_OnPrem','GCLL-0257-T-02', 'GCLL-0257-T-03']

In [5]:
len(sample_list)

6

In [6]:
# Output file

In [7]:
outputfile_name = "CLL_subplot_combined_0257_wbc_updated.html"

In [8]:


# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    abundance_df = pd.read_csv(participants.loc[patient_id]['abundances_tsv'], sep='\t')
    mcmc_df = pd.read_csv(participants.loc[patient_id]['cell_population_mcmc_trace'], sep='\t')
    tree_df = pd.read_csv(participants.loc[patient_id]['tree_tsv'], sep='\t')
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    sample_list = wbc_df_patient['Timepoint'].dropna().tolist()
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Time_since_start_tx'].tolist()]
    wbc_all = [float(i) for i in wbc_df_patient['WBC'].tolist()]
    
    
    CLL_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL count estm'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Time_since_start_tx'].to_list()]
    
    return wbc_df_patient, times_sample, CLL_count_sample, wbc_all,all_times





Update patient specific information in the below cell 

In [9]:
end_treatment_df = pd.read_csv("../../../data/gcll/CLL8_end_treatment.csv")

In [10]:
# Get the treatment duration 

treatment_end = end_treatment_df[end_treatment_df.Patient == patient_id].Treatment_duration.iloc[0]

In [11]:
treatment_end

139

In [12]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file, treatment_file)

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [13]:
times_sample

[-8, 82, 437, 731, 852, 1407]

In [14]:
wbc_count

[13.3, 5.9, 5.0, 3.6, 4.1, 4.8, 4.1, 9.9, 11.7, 16.7, 20.5, 509.0]

In [15]:
CLL_count_sample

[10.64, 0.12033502, 0.17149882, 1.98628973, 12.915, 498.82]

In [16]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [17]:
 plot_tree_html = plot_tree_plotly(tree_df, 0)

In [18]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [45]:
branch_annotations = {
    1: "SF3B1_p.G742D, del(13q), del(14q32)",
    2: "SF3B1_p.Y623C,ZC3H18_p.R402fs,amp(2p)", 
    3:"IKZF3_p.L162R"
}

In [46]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=0,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

SF3B1_p.G742D<br>del(13q)<br>del(14q32)
SF3B1_p.Y623C<br>ZC3H18_p.R402fs<br>amp(2p)


In [20]:
CLL_count_sample

[10.64, 0.12033502, 0.17149882, 1.98628973, 12.915, 498.82]

In [21]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [22]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [23]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [24]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [25]:
times_sample

[-8, 82, 437, 731, 852, 1407]

In [26]:
times_aft_tx

[139, 166, 215, 311, 437, 532, 649, 731, 831, 852, 1407]

In [27]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [28]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [29]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_79216/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
times_aft_tx

[139, 166, 215, 311, 437, 532, 649, 731, 831, 852, 1407]

In [31]:
# Needs to manually check times_sliced_aft

In [32]:
times_sliced_aft

[82, 166, 215, 311, 437, 532, 649, 731, 831, 852, 1407]

In [33]:
times_sliced_aft = times_sliced_aft[1:]

In [34]:
len(times_sliced_aft)

10

In [35]:
index_samples_model = slice(2,6)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [36]:
log_subclone_sample_mcmc_with_uniform_noise[1][2]

[1.1520940638580135,
 -3.4609863429310828,
 -2.594897394710939,
 -0.40154475744091755,
 -0.43443162719694123,
 4.181256136711909]

In [37]:
y

[None,
 None,
 None,
 array([-2.59489739, -2.98620516, -3.79310926, -3.83550197, -6.05912326]),
 None,
 None,
 array([-0.40154476, -0.28808032, -1.18761635, -1.51528581, -3.13261824]),
 None,
 array([-0.43443163,  1.68362255,  1.26074027,  1.12742141, -1.29799034]),
 array([4.18125614, 3.01369446, 4.98339571, 5.52166204, 2.8471981 ])]

In [38]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [39]:
wbc_model

[0.00813511,
 0.01313868,
 0.0216232,
 0.17149882,
 0.24796375,
 1.00003946,
 1.98628973,
 6.40480512,
 12.915,
 498.82]

In [40]:
len(wbc_model)

10

In [41]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [42]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [43]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)

In [47]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as CLL_subplot_combined_0257_wbc_updated.html
